# MIDI Exploration

Inspiration taken from [this tutorial](https://nbviewer.jupyter.org/github/craffel/midi-dataset/blob/master/Tutorial.ipynb).

We use a reduced and cleaned Lakh MIDI dataset [LMD](https://colinraffel.com/projects/lmd/) for this exploration.

It is assumed that these files are extracted in a directory called "clean midi" in the folder one level above this (../clean_midi).

In [ ]:
#!pip install pretty-midi librosa pandas

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pretty_midi
import librosa.display as display
import os
import pandas as pd

# Local path constants
DATA_PATH = '../clean_midi'

In [ ]:
song_name = "Head Like a Hole.mid"

pm = pretty_midi.PrettyMIDI(os.path.join(DATA_PATH, "Nine Inch Nails/{}".format(song_name)))

piano_roll = pm.get_piano_roll()

plt.figure(figsize=(10, 6))
display.specshow(piano_roll, y_axis='cqt_note', cmap=plt.cm.hot)
plt.title(song_name)

# Get instrument names from midi

In [ ]:
for instrument in pm.instruments:
    print(instrument)

# Get notes for specific instrument

In [ ]:
instrument_no = 3

instrument = pm.instruments[instrument_no]

cols = ['start', 'end', 'pitch', 'velocity']

note_seq = []
for index, note in enumerate(instrument.notes):
    note_seq.append([note.start, note.end, note.pitch, note.velocity])
    
note_df = pd.DataFrame(note_seq, columns=cols)
note_df.head()

# Or as a numpy array

In [ ]:
note_seq_np = np.zeros((len(instrument.notes), 4))
for index, note in enumerate(instrument.notes):
    note_seq_np[index] = [note.start, note.end, note.pitch, note.velocity]

note_seq_np

# Attribution and References

- Colin Raffel. "Learning-Based Methods for Comparing Sequences, with Applications to Audio-to-MIDI Alignment and Matching". [PhD Thesis, 2016](http://colinraffel.com/publications/thesis.pdf).

- Colin Raffel and Daniel P. W. Ellis. [Intuitive Analysis, Creation and Manipulation of MIDI Data with pretty_midi](http://colinraffel.com/publications/ismir2014intuitive.pdf). In Proceedings of the 15th International Conference on Music Information Retrieval Late Breaking and Demo Papers, 2014.

- [Librosa](https://doi.org/10.5281/zenodo.591533): Audio and music signal analysis in python
